In [1]:
# Install required packages
!pip install transformers accelerate torch datasets trl peft bitsandbytes
!pip install comet-ml unbabel-comet bert-score sacrebleu
!pip install wandb pandas numpy tqdm

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 556.4/556.4 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.4/59.4 MB 29.5 MB/s eta 0:00:00m eta 0:00:010:01:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.7/47.7 MB 26.7 MB/s eta 0:00:00m eta 0:00:010:00:01
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 19.0.0
    Uninstalling pyarrow-19.0.0:
      Successfully uninstalled pyarrow-19.0.0
  Attempting uninstall: dill0m╺━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1/8 [pyarrow]
    Found existing installation: dill 0.3.8━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1/8 [pyarrow]
    Uninstalling dill-0.3.8:38;5;237m╺━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1/8 [pyarrow]
      Successfully uninstalled dill-0.3.8m━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1/8 [pyarrow]
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8/8 [peft]7m━━━━ 7/8 [peft]ndbytes]
  Using cached numpy-1.26.4-cp313-cp313-linux_x86_64.whl
  Attempting uninstall: numpy
    Found existing installation: numpy 2.2.6
 

In [2]:
import os
import torch
from pathlib import Path

# Check GPU availability
print(f"GPU Available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    num_gpus = torch.cuda.device_count()
    print(f"Number of GPUs: {num_gpus}")
    for i in range(num_gpus):
        print(f"  GPU {i}: {torch.cuda.get_device_name(i)}")
        print(f"    Memory: {torch.cuda.get_device_properties(i).total_memory / 1e9:.2f} GB")
    
    # Total available VRAM
    total_vram = sum(torch.cuda.get_device_properties(i).total_memory for i in range(num_gpus))
    print(f"\nTotal VRAM: {total_vram / 1e9:.2f} GB")
else:
    print("⚠️ No GPU detected! Training will be very slow.")

GPU Available: True
Number of GPUs: 2
  GPU 0: NVIDIA GeForce RTX 5090
    Memory: 33.67 GB
  GPU 1: NVIDIA GeForce RTX 5090
    Memory: 33.67 GB

Total VRAM: 67.34 GB


## Project Configuration

In [3]:
from pathlib import Path
import os
from huggingface_hub import login, hf_hub_download

# ===========================
# VIRTUAL ENV DETECTION
# ===========================
VENV_PATH = os.environ.get("VIRTUAL_ENV")
if VENV_PATH:
    print(f"Using virtual environment at: {VENV_PATH}")
else:
    print("Warning: Not running inside a virtual environment!")

# ===========================
# HUGGING FACE AUTH
# ===========================
# Uses token saved in ~/.cache/huggingface/token
HF_TOKEN_PATH = Path.home() / ".cache/huggingface/token"
if HF_TOKEN_PATH.exists():
    with open(HF_TOKEN_PATH) as f:
        hf_token = f.read().strip()
    os.environ["HUGGINGFACE_HUB_TOKEN"] = hf_token
    print("Hugging Face token loaded from cache.")
else:
    print("No Hugging Face token found. Please run `huggingface-cli login`.")

# ===========================
# PROJECT PATHS (Auto-detect from notebook location)
# ===========================
# Try to auto-detect the project directory from the notebook's location
_notebook_dir = Path(__file__).parent if '__file__' in dir() else Path.cwd()

# Check common locations
_possible_paths = [
    Path("/home/aya/Desktop/ENSIA 4Y/S1/NLP/Project/Context_Specific_Machine_Translation_to_Arabic_Language/Reinforcement Learning"),
    _notebook_dir,
    Path.cwd(),
    Path.home() / "Downloads" / "Reinforcement Learning",
]

PROJECT_DIR = None
for _path in _possible_paths:
    if (_path / "data").exists() or (_path / "0_config_setup.ipynb").exists():
        PROJECT_DIR = _path
        break

if PROJECT_DIR is None:
    PROJECT_DIR = _possible_paths[0]  # Default to first path
    print(f"⚠️ Using default project path: {PROJECT_DIR}")
else:
    print(f"✅ Detected project directory: {PROJECT_DIR}")

DATA_DIR = PROJECT_DIR / "data"
MODELS_DIR = PROJECT_DIR / "models"
OUTPUTS_DIR = PROJECT_DIR / "outputs"
LOGS_DIR = PROJECT_DIR / "logs"

for dir_path in [DATA_DIR, MODELS_DIR, OUTPUTS_DIR, LOGS_DIR]:
    dir_path.mkdir(exist_ok=True, parents=True)
    
print(f"📁 Data directory: {DATA_DIR}")
print(f"📁 Models directory: {MODELS_DIR}")

# ===========================
# TRANSLATION DATA PATH (for 2M sentences dataset)
# ===========================
# Path to your CSV with source/target columns
TRANSLATION_DATA_PATH = PROJECT_DIR.parent / "Data" / "english-arabic"
if TRANSLATION_DATA_PATH.exists():
    _csv_files = list(TRANSLATION_DATA_PATH.glob("*.csv"))
    print(f"✅ Found translation data: {len(_csv_files)} CSV files")
    for f in _csv_files:
        print(f"   - {f.name}")
else:
    print(f"⚠️ Translation data not found at: {TRANSLATION_DATA_PATH}")

# ===========================
# MODEL PATHS
# ===========================
# Fine-tuned GemmaX2-28-9B (will download automatically if not present)
SFT_MODEL_NAME = "google/gemma-2-9b-it"  # Use Hugging Face model name
SFT_MODEL_PATH = MODELS_DIR / "GemmaX2-28-9B"

# Create local cache folder
SFT_MODEL_PATH.mkdir(exist_ok=True, parents=True)

# Base model for reward model
REWARD_BASE_MODEL = "google/gemma-2-2b"  # Or local path if downloaded

# ===========================
# DATA FILES
# ===========================
TEST_PROMPTS = DATA_DIR / "test_prompts.jsonl"
SYNTHETIC_PREFERENCES = DATA_DIR / "synthetic_preferences.jsonl"
HUMAN_PREFERENCES = DATA_DIR / "human_preferences.jsonl"
GENERATED_CANDIDATES = DATA_DIR / "generated_candidates.jsonl"

# ===========================
# MODEL CHECKPOINTS
# ===========================
REWARD_MODEL_COLD_START = MODELS_DIR / "reward_model_coldstart"
REWARD_MODEL_HUMAN_ALIGNED = MODELS_DIR / "reward_model_human"
PPO_MODEL_COLD_START = MODELS_DIR / "ppo_model_coldstart"
PPO_MODEL_FINAL = MODELS_DIR / "ppo_model_final"

print("✅ Directory structure and model paths are ready!")

No Hugging Face token found. Please run `huggingface-cli login`.
✅ Detected project directory: /home/aya/Desktop/Context_Specific_Machine_Translation_to_Arabic_Language/Reinforcement Learning
📁 Data directory: /home/aya/Desktop/Context_Specific_Machine_Translation_to_Arabic_Language/Reinforcement Learning/data
📁 Models directory: /home/aya/Desktop/Context_Specific_Machine_Translation_to_Arabic_Language/Reinforcement Learning/models
✅ Found translation data: 2 CSV files
   - technology.csv
   - Education_Research.csv
✅ Directory structure and model paths are ready!


## Hyperparameters

In [8]:
# ===========================
# HPC CONFIGURATION (2x NVIDIA 5090)
# ===========================

# Multi-GPU settings
NUM_GPUS = torch.cuda.device_count() if torch.cuda.is_available() else 0
USE_MULTI_GPU = NUM_GPUS > 1
GPU_MEMORY_PER_DEVICE = "30GB"  # Leave headroom on each 5090

# Use Flash Attention 2 for faster inference (if available)
USE_FLASH_ATTENTION = False  # Changed to False - flash-attn package not installed
ATTN_IMPLEMENTATION = "flash_attention_2" if USE_FLASH_ATTENTION else "eager"

print(f"HPC Config: {NUM_GPUS} GPUs, Flash Attention: {USE_FLASH_ATTENTION}")

# ===========================
# COMET SCORING CONFIGURATION
# ===========================

# COMET model for quality estimation (reference-free)
COMET_MODEL = "Unbabel/wmt22-cometkiwi-da"  # QE model - no reference needed
# Alternative models:
#   "Unbabel/wmt22-comet-da" - requires reference translation
#   "Unbabel/wmt20-comet-da" - older but faster
#   "Unbabel/XCOMET-XL" - larger, more accurate

COMET_BATCH_SIZE = 64  # Batch size for COMET scoring (adjust based on GPU memory)
COMET_GPU_ID = 1 if NUM_GPUS > 1 else 0  # Use second GPU for COMET if available

# Scoring weights (COMET-primary)
METRIC_WEIGHTS = {
    'comet': 0.70,      # Main quality metric (reference-free)
    'length': 0.15,     # Length ratio heuristic
    'arabic': 0.15      # Arabic character ratio
}

print(f"COMET Config: model={COMET_MODEL}, batch_size={COMET_BATCH_SIZE}, gpu={COMET_GPU_ID}")

# ===========================
# SYNTHETIC DATA GENERATION
# ===========================

NUM_CANDIDATES = 4  # Number of translation variants per input (reduced for speed with COMET)
BATCH_SIZE_GENERATION = 16  # Process multiple sources at once (optimized for 2x 5090)
MEGA_BATCH_SIZE = 64  # Mega-batch for parallel generation
SAMPLE_SIZE = 100000  # Number of samples to process (from 2M dataset)

TEMPERATURES = [0.6, 0.8, 1.0, 1.2]  # Temperature variations
TOP_K_VALUES = [30, 50, 80]
TOP_P_VALUES = [0.85, 0.9, 0.95]

# Generation parameters
MAX_NEW_TOKENS = 256
USE_BFLOAT16 = True
FORCE_CPU = False  # Set to True to force CPU (for debugging)

# ===========================
# REWARD MODEL TRAINING (Optimized for 64GB VRAM)
# ===========================

RM_LEARNING_RATE = 1e-5
RM_BATCH_SIZE = 32  # Increased from 8 for better GPU utilization
RM_EPOCHS = 3
RM_MAX_LENGTH = 512
RM_GRADIENT_ACCUMULATION_STEPS = 2  # Reduced since batch size is larger

# Reward head architecture
RM_HEAD_TYPE = "mlp"  # "linear" or "mlp"
RM_HIDDEN_DIM = 256  # Only used if mlp

# ===========================
# PPO TRAINING (Optimized for 64GB VRAM)
# ===========================

PPO_LEARNING_RATE = 1.41e-5
PPO_BATCH_SIZE = 32  # Increased from 8
PPO_MINI_BATCH_SIZE = 8  # Increased from 2
PPO_GRADIENT_ACCUMULATION_STEPS = 2  # Reduced from 4
PPO_EPOCHS = 1
PPO_STEPS = 1000  # Total optimization steps

# PPO specific parameters - INCREASED KL for safety
KL_PENALTY_COEF = 0.15  # Increased from 0.1 for better preservation of SFT weights
KL_TARGET = 0.1  # Target KL divergence
MAX_KL_THRESHOLD = 0.2  # Stop if KL exceeds this (safety)
CLIP_RANGE = 0.2
VALUE_CLIP_RANGE = 0.2
GAE_LAMBDA = 0.95
GAMMA = 0.99

# Generation parameters for PPO
PPO_MAX_NEW_TOKENS = 256
PPO_TEMPERATURE = 0.9

# ===========================
# LoRA CONFIGURATION (Optional - for safer training)
# ===========================

USE_LORA = True  # Set to True to use LoRA (recommended for preserving SFT weights)
LORA_R = 16
LORA_ALPHA = 32
LORA_DROPOUT = 0.05
LORA_TARGET_MODULES = ["q_proj", "v_proj", "k_proj", "o_proj"]

# ===========================
# INFERENCE
# ===========================

INFERENCE_NUM_CANDIDATES = 8  # Generate 5-8 candidates
INFERENCE_TOP_K = 3  # Show top 3 to user
INFERENCE_MAX_LENGTH = 512

# ===========================
# GENERAL SETTINGS
# ===========================

USE_WANDB = True  # Set to True if using Weights & Biases for tracking
WANDB_PROJECT = "rlhf-arabic-translation"
SEED = 42

# ===========================
# EARLY STOPPING / MONITORING
# ===========================

QUALITY_DEGRADATION_THRESHOLD = 0.05  # Stop if quality drops by 5%
CHECKPOINT_EVERY_N_STEPS = 100

print("✅ Configuration loaded successfully!")
print(f"   - COMET enabled for scoring")
print(f"   - LoRA enabled: {USE_LORA}")
print(f"   - KL penalty: {KL_PENALTY_COEF}")
print(f"   - Sample size: {SAMPLE_SIZE:,}")
print(f"   - Batch sizes: Generation={MEGA_BATCH_SIZE}, RM={RM_BATCH_SIZE}, PPO={PPO_BATCH_SIZE}")

HPC Config: 2 GPUs, Flash Attention: False
COMET Config: model=Unbabel/wmt22-cometkiwi-da, batch_size=64, gpu=1
✅ Configuration loaded successfully!
   - COMET enabled for scoring
   - LoRA enabled: True
   - KL penalty: 0.15
   - Sample size: 100,000
   - Batch sizes: Generation=64, RM=32, PPO=32


In [5]:
import random
import numpy as np

def set_seed(seed=42):
    """Set random seeds for reproducibility"""
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)

def load_test_prompts(file_path):
    """Load test prompts from JSONL file
    
    Expected format: one JSON object per line
    {"text": "source_text", "lang": "en" or "fr"}
    """
    import json
    data = []
    with open(file_path, 'r', encoding='utf-8') as f:
        for line_num, line in enumerate(f, 1):
            line = line.strip()
            # Skip empty lines and comments
            if not line or line.startswith('//') or line.startswith('#'):
                continue
            try:
                item = json.loads(line)
                data.append({
                    'source': item['text'],
                    'source_lang': item.get('lang', 'en')
                })
            except json.JSONDecodeError as e:
                print(f"Warning: Skipping invalid JSON on line {line_num}: {e}")
                continue
    return data

def format_translation_prompt(text, source_lang='en'):
    """Format input text as translation prompt"""
    lang_name = {'en': 'English', 'fr': 'French'}[source_lang]
    return f"Translate the following {lang_name} text to Arabic:\n\n{text}\n\nArabic translation:"

set_seed(SEED)
print("Utility functions loaded!")

Utility functions loaded!


## Save Configuration

In [6]:
import json

# Save configuration to JSON for reference
config = {
    'sft_model_path': str(SFT_MODEL_PATH),
    'reward_base_model': REWARD_BASE_MODEL,
    'num_candidates': NUM_CANDIDATES,
    'metric_weights': METRIC_WEIGHTS,
    'rm_learning_rate': RM_LEARNING_RATE,
    'rm_batch_size': RM_BATCH_SIZE,
    'rm_epochs': RM_EPOCHS,
    'ppo_learning_rate': PPO_LEARNING_RATE,
    'ppo_steps': PPO_STEPS,
    'kl_penalty_coef': KL_PENALTY_COEF,
    'seed': SEED
}

config_path = PROJECT_DIR / "config.json"
with open(config_path, 'w') as f:
    json.dump(config, f, indent=2)

print(f"Configuration saved to {config_path}")

Configuration saved to /home/aya/Desktop/Context_Specific_Machine_Translation_to_Arabic_Language/Reinforcement Learning/config.json


## Next Steps

1. **Update the `SFT_MODEL_PATH`** to point to your Gemma-2X289B model location
2. **Prepare your data**: Place EN-AR and FR-AR parallel corpora in the data directory
3. **Run notebook 1**: Generate synthetic preference data
4. **Run notebook 2**: Train the reward model
5. **Run notebook 3**: Run PPO optimization
6. **Run notebook 4**: Test inference and collect user feedback
7. **Run notebook 5**: Fine-tune with human preferences